In [1]:
import pandas as pd
import matplotlib as mp
import numpy as np
from datetime import timedelta
import datetime as dt
import gc

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb

%matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Levanto los installs ya procesados
....

In [2]:
#Root de archivos de datos
root_dir = '../data/'

#Lee menos data si es testing
is_testing = False

# Cantidad minima de aparicion de equipos en un dataframe
#min_devices = 5
min_devices = 20

# Identificador unico de equipos
device_uuid = ['ref_hash']

# Ventanas de tiempo
windows = pd.DataFrame({
    'begin_date': [dt.datetime(2019,4,18),dt.datetime(2019,4,21),dt.datetime(2019,4,24)],
    'window_nr':[1,2,3]
})

In [3]:
installs_dtypes = {
    'application_id': np.int32,
    'ref_hash': np.int64, 
    #'click_hash':'category',
    'attributed': 'category',
    'implicit': 'category',
    #'device_countrycode': 'object', 
    'device_brand': 'object',
    'device_model': 'object', 
    'session_user_agent': 'object', 
    'user_agent': 'object', 
    'event_uuid':'object',
    'kind': 'object',
    'wifi': 'object', 
    'trans_id': 'object', 
    #'ip_address':'object', 
    'device_language': 'object'
}
install_cols = list(installs_dtypes.keys()) + ['created']
installs = pd.read_csv(root_dir + 'installs.csv.gzip', 
                       compression='gzip', 
                       usecols=install_cols,
                       dtype= installs_dtypes,
                      parse_dates=['created'])

installs['n'] = 1
print('#records install:', installs['n'].sum())

('#records install:', 481511)


In [4]:
# Labels a submitir con las predicciones
to_predict = pd.read_csv(root_dir + 'target_competencia_ids.csv')
to_predict = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
to_predict.columns = ['ref_hash']

In [5]:
unique_hashes = pd.read_csv(root_dir+'unique_hashes.csv')

---
## 2. Arreglo de los datos

### Separo las semanas de entrenamiento

Se utiliza el siguiente metodo: 

1. Ventana del 18 al 20 inclusive (1) -> Predice valores entre el 21 y 24 (2)
2. Ventana del 21 al 23 inclusive (2) -> Predice valores entre el 24 y 26 (3)

In [6]:
print("Tiempo init: "+str(pd.Timestamp.now()))

Tiempo init: 2019-06-23 17:12:00.079709


In [7]:
installs = installs.loc[installs['ref_hash'].isin(unique_hashes['ref_hash'])]
installs.sort_values(by='created',inplace=True)
installs = pd.merge_asof(installs,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

### 2.2 Arreglo de datos de Installs

In [8]:
installs['kind'] = installs['kind'].str.replace(' ','_')
installs['kind'] = installs['kind'].str.replace('af_app_open ','af_app_opened')
installs['kind'] = installs['kind'].str.replace('af_app_opend','af_app_opened')
installs['kind'] = installs['kind'].str.lower()

In [9]:
installs.isnull().sum()

created                    0
application_id             0
ref_hash                   0
attributed                 0
implicit                   0
device_brand          204813
device_model           26871
session_user_agent     14828
user_agent            150111
event_uuid            377704
kind                  377704
wifi                  186016
trans_id              472140
device_language        27552
n                          0
window_nr                  0
dtype: int64

In [10]:
for i in ['device_brand','device_model','session_user_agent','user_agent','kind','wifi','device_language']:
    installs[i] = installs[i].fillna('unknown')
installs['device_brand'] = installs['device_brand'].astype('category')
installs['device_model'] = installs['device_model'].astype('category')
installs['session_user_agent'] = installs['session_user_agent'].astype('category')
installs['user_agent'] = installs['user_agent'].astype('category')
installs['kind'] = installs['kind'].astype('category')
installs['wifi'] = installs['wifi'].astype('category')
installs['device_language'] = installs['device_language'].astype('category')

In [11]:
installs.head().transpose()

,0,1,2,3,4
created,2019-04-18 00:00:01.560000,2019-04-18 00:00:01.851000,2019-04-18 00:00:05.152000,2019-04-18 00:00:05.589000,2019-04-18 00:00:06.795000
application_id,70,70,65,27,339
ref_hash,4432995619177048534,5904733559638204455,896373747754111825,3399210824535017892,1541425881979513687
attributed,False,False,False,False,False
implicit,False,False,True,False,False
device_brand,unknown,unknown,3.083058605577787e+17,unknown,unknown
device_model,unknown,unknown,5.274185305862703e+18,6.794880020077885e+18,6.794880020077885e+18
session_user_agent,Apsalar-Postback,Apsalar-Postback,http-kit/2.0,http-kit/2.0,http-kit/2.0
user_agent,unknown,unknown,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0
event_uuid,NaN,NaN,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,NaN,NaN


In [12]:
print("Tiempo end: "+str(pd.Timestamp.now()))

Tiempo end: 2019-06-23 17:12:14.932552


In [13]:
installs.head(1)

,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n,window_nr
0,2019-04-18 00:00:01.560,70,4432995619177048534,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1


### Cant. de Apps distintas instaladas	por ref_hash

In [14]:
#created: fecha de instalacion de app
#application_id: id interno de instalacion de app
installs['n'] = 1
inst_app_count = installs.groupby(by=['ref_hash','application_id']).sum()
inst_app_count = inst_app_count.reset_index()

In [15]:
inst_app_count.head()

,ref_hash,application_id,n,window_nr
0,40621409780134,77,1,2
1,41863526108385,65,2,2
2,41863526108385,121,2,2
3,90072729247980,210,1,3
4,135153013040192,155,1,1


In [16]:
inst_app_count['m'] = 1

In [17]:
inst_app_count.head(5)

,ref_hash,application_id,n,window_nr,m
0,40621409780134,77,1,2,1
1,41863526108385,65,2,2,1
2,41863526108385,121,2,2,1
3,90072729247980,210,1,3,1
4,135153013040192,155,1,1,1


In [18]:
temp = inst_app_count.groupby('ref_hash').sum()['m'].reset_index()
temp.head()

,ref_hash,m
0,40621409780134,1
1,41863526108385,2
2,90072729247980,1
3,135153013040192,1
4,161514654074162,1


In [19]:
inst_app_count = inst_app_count.merge(temp,how='inner',on='ref_hash')
inst_app_count = inst_app_count[['ref_hash','m_y','window_nr']]
inst_app_count = inst_app_count.rename(columns={"m_y":"count_app_dif"})
inst_app_count = inst_app_count.drop_duplicates()

inst_app_count = inst_app_count.sort_values(by='count_app_dif',ascending=False)
inst_app_count = inst_app_count.drop_duplicates()
inst_app_count.head()

,ref_hash,count_app_dif,window_nr
247023,5446085605337844584,10,2
247027,5446085605337844584,10,4
41177,906973248467925335,9,4
41178,906973248467925335,9,1
41171,906973248467925335,9,2


In [20]:
inst_app_count.head(2)

,ref_hash,count_app_dif,window_nr
247023,5446085605337844584,10,2
247027,5446085605337844584,10,4


# Feature: Instaló en la ventana anterior ?

In [44]:
installs.loc[installs['window_nr'] > 3]

,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n,window_nr


In [45]:
inst_app_count.head(6)

,ref_hash,count_app_dif,window_nr
247023,5446085605337844584,10,2
247027,5446085605337844584,10,4
41177,906973248467925335,9,4
41178,906973248467925335,9,1
41171,906973248467925335,9,2
287444,6337720443035762926,9,1


In [46]:
in_sort2 = in_sort.merge(win_ok,how='inner',on='ref_hash')
in_sort2 = in_sort2[['ref_hash','count_app_dif','window_nr_y']]
in_sort2 = in_sort2.rename(columns={'window_nr_y':'window_nr'})
in_sort2.head(3)

NameError: name 'in_sort' is not defined

In [ ]:
in_sort2.loc[in_sort2['window_nr'] > 3]

In [ ]:
in_sort.loc[in_sort['window_nr'] > 3]

In [ ]:
in_sort = in_sort2

In [ ]:
in_sort = inst_app_count.sort_values(by=['count_app_dif','ref_hash','window_nr'],ascending=False)
in_sort.head(4)

# Feature: Instaló en la ventana anterior ?

In [47]:
installs.loc[installs['window_nr'] > 3]

,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n,window_nr


In [48]:
in_next_prev = installs[['ref_hash','window_nr']]
in_next_prev.head(5)
print("Count: "+str(in_next_prev.count()))

Count: ref_hash     480566
window_nr    480566
dtype: int64


In [49]:
ref_win_1 = in_next_prev.loc[in_next_prev['window_nr'] == 1][['ref_hash']]
ref_win_2 = in_next_prev.loc[in_next_prev['window_nr'] == 2][['ref_hash']]
ref_win_3 = in_next_prev.loc[in_next_prev['window_nr'] == 3][['ref_hash']]
ref_win_2['win_2_in_1'] = ref_win_2['ref_hash'].isin(ref_win_1['ref_hash'])
ref_win_3['win_3_in_2'] = ref_win_3['ref_hash'].isin(ref_win_2['ref_hash'])
in_win = in_next_prev.merge(ref_win_2,how="left",on="ref_hash")
in_win = in_win.merge(ref_win_3,how="inner",on="ref_hash")
in_win_d = in_win.drop_duplicates()
in_win_d.head(10)

,ref_hash,window_nr,win_2_in_1,win_3_in_2
0,8509930510698276342,1,NaN,False
2,8509930510698276342,3,NaN,False
3,5072620966954264118,1,True,True
5,5072620966954264118,2,True,True
7,5072620966954264118,3,True,True
11,3664457589760889308,1,NaN,False
12,3664457589760889308,3,NaN,False
13,5894856328487209314,1,True,True
14,5894856328487209314,2,True,True
15,5894856328487209314,3,True,True


In [50]:
in_win_e = in_win_d.fillna(False)
in_win_e.head(10)

,ref_hash,window_nr,win_2_in_1,win_3_in_2
0,8509930510698276342,1,False,False
2,8509930510698276342,3,False,False
3,5072620966954264118,1,True,True
5,5072620966954264118,2,True,True
7,5072620966954264118,3,True,True
11,3664457589760889308,1,False,False
12,3664457589760889308,3,False,False
13,5894856328487209314,1,True,True
14,5894856328487209314,2,True,True
15,5894856328487209314,3,True,True


In [51]:
in_win_e.loc[in_win_e['window_nr'] > 3]

,ref_hash,window_nr,win_2_in_1,win_3_in_2


In [55]:
in_win_e = in_win_e.sort_values(by='ref_hash')

In [60]:
in_win_f = in_win_e.reset_index()[['ref_hash','window_nr','win_2_in_1','win_3_in_2']]
in_win_f = in_win_f.sort_values(by='ref_hash')
in_win_f.head()

,ref_hash,window_nr,win_2_in_1,win_3_in_2
0,90072729247980,3,False,False
1,342614246084071,3,False,False
2,347785260789835,3,False,False
3,416301579449694,3,False,False
4,420449720220692,3,False,False


In [61]:
in_win_f.to_csv('/tmp/in_win.csv')

# Feature: Cantidad de eventos por tipo
### Para cada tipo en vez de colocar la categoria seteo la cantidad de ese tipo para todo el dataframe.


In [62]:
installs.head()

,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n,window_nr
0,2019-04-18 00:00:01.560,70,4432995619177048534,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1
1,2019-04-18 00:00:01.851,70,5904733559638204455,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1
2,2019-04-18 00:00:05.152,65,896373747754111825,False,True,3.083058605577787e+17,5.274185305862703e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,af_app_opened,false,NaN,6.977049253562486e+18,1,1
3,2019-04-18 00:00:05.589,27,3399210824535017892,False,False,unknown,6.794880020077885e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,unknown,true,NaN,5.221862722669226e+18,1,1
4,2019-04-18 00:00:06.795,339,1541425881979513687,False,False,unknown,6.794880020077885e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,unknown,true,NaN,7.528973756559112e+18,1,1


In [67]:
kind_count = installs.groupby('kind').sum().reset_index()[['kind','n']]
kind_count.head()

,kind,n
0,0-1,2.0
1,01_launch,33.0
2,1st_ride,139.0
3,abertura_do_app,34.0
4,account_summary_first_step,504.0


In [68]:
inst_kind = installs.copy()

In [69]:
inst_kind2 = inst_kind.merge(kind_count,how='inner',on='kind')
inst_kind2.head()

,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n_x,window_nr,n_y
0,2019-04-18 00:00:01.560,70,4432995619177048534,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1,377704.0
1,2019-04-18 00:00:01.851,70,5904733559638204455,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1,377704.0
2,2019-04-18 00:00:05.589,27,3399210824535017892,False,False,unknown,6.794880020077885e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,unknown,true,NaN,5.221862722669226e+18,1,1,377704.0
3,2019-04-18 00:00:06.795,339,1541425881979513687,False,False,unknown,6.794880020077885e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,unknown,true,NaN,7.528973756559112e+18,1,1,377704.0
4,2019-04-18 00:00:08.994,210,8942039642364169230,False,False,unknown,2.0193221952374024e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; FIG-LX3...,NaN,unknown,unknown,NaN,3.3013777759776993e+18,1,1,377704.0


In [70]:
inst_kind = inst_kind2.rename(columns={"n_y":"count_for_kind"})
inst_kind.head(2)

,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n_x,window_nr,count_for_kind
0,2019-04-18 00:00:01.560,70,4432995619177048534,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1,377704.0
1,2019-04-18 00:00:01.851,70,5904733559638204455,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1,377704.0


In [71]:
inst_kind_train = inst_kind[['ref_hash','count_for_kind']]
inst_kind_train.head()

,ref_hash,count_for_kind
0,4432995619177048534,377704.0
1,5904733559638204455,377704.0
2,3399210824535017892,377704.0
3,1541425881979513687,377704.0
4,8942039642364169230,377704.0


In [72]:
inst_kind_train.tail()

,ref_hash,count_for_kind
480561,5881015695987735476,4.0
480562,8479641616954333258,1.0
480563,6238227029646471100,1.0
480564,144858825658199154,1.0
480565,7524575935906870202,1.0


# Feature: device_language count
### cambio el hash del 'device_language' por la cantidad de apariciones que tiene en el dataframe 

In [23]:
installs.head(2)

,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n,window_nr
0,2019-04-18 00:00:01.560,70,4432995619177048534,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1
1,2019-04-18 00:00:01.851,70,5904733559638204455,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1


In [25]:
installs['device_language'].head()
installs['ref_hash'].head()

0    4432995619177048534
1    5904733559638204455
2     896373747754111825
3    3399210824535017892
4    1541425881979513687
Name: ref_hash, dtype: int64

In [27]:
installs.groupby('ref_hash').sum().head()

,application_id,n,window_nr
ref_hash,,,
40621409780134,77,1,2
41863526108385,372,4,4
90072729247980,210,1,3
135153013040192,155,1,1
161514654074162,65,1,2


In [28]:
#in2 = installs.iloc[1:1000,:]
in2 = installs.copy()

in2.head(2)

,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n,window_nr
0,2019-04-18 00:00:01.560,70,4432995619177048534,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1
1,2019-04-18 00:00:01.851,70,5904733559638204455,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1


In [29]:
in3 = in2.groupby('device_language').count().reset_index()[['device_language','n']]
in3.head()

,device_language,n
0,1.0422435784653747e+18,1
1,1.0451662391819459e+18,16
2,1.1074169494706739e+18,1
3,1.1251792116984398e+18,2
4,1.1634527299348777e+18,21


In [30]:
in4 = in3.merge(in2,how='inner',on='device_language')
in4.head()

,device_language,n_x,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,n_y,window_nr
0,1.0422435784653747e+18,1,2019-04-18 02:04:37.731,68,6056108421961886671,False,False,unknown,6.794880020077885e+18,http-kit/2.0,Grability/17420 CFNetwork/978.0.7 Darwin/18.5.0,NaN,unknown,true,NaN,1,1
1,1.0451662391819459e+18,16,2019-04-18 02:52:48.543,68,7920687496628265608,False,False,unknown,6.794880020077885e+18,http-kit/2.0,Grability/17420 CFNetwork/902.2 Darwin/17.7.0,NaN,unknown,false,NaN,1,1
2,1.0451662391819459e+18,16,2019-04-18 21:23:27.785,194,6091860428758048147,False,False,unknown,9.173806934258561e+18,http-kit/2.0,Gardenscapes/3.3.0 CFNetwork/811.5.4 Darwin/16...,NaN,unknown,true,NaN,1,1
3,1.0451662391819459e+18,16,2019-04-19 04:25:21.676,4,7149457689091958851,False,False,unknown,6.794880020077885e+18,http-kit/2.0,Cornershop/539 CFNetwork/978.0.7 Darwin/18.5.0,NaN,unknown,true,NaN,1,1
4,1.0451662391819459e+18,16,2019-04-19 20:41:14.289,198,3436307134849304427,False,False,unknown,6.794880020077885e+18,http-kit/2.0,shpock/88201 CFNetwork/978.0.7 Darwin/18.5.0,NaN,unknown,true,NaN,1,1


In [31]:
in4 = in4[['ref_hash','device_language','n_x']]
in4['dev_lang_count'] = in4['n_x']
in4 = in4[['ref_hash','dev_lang_count']]
in4.head()

,ref_hash,dev_lang_count
0,6056108421961886671,1
1,7920687496628265608,16
2,6091860428758048147,16
3,7149457689091958851,16
4,3436307134849304427,16


In [32]:
#in2 = in2.groupby(by=['ref_hash']).sum().reset_index()
#in2.sort_values('n').head()

#in2 = installs.groupby('device_language').sum().reset_index()
#in2.sort_values(by='n',ascending=False)
#in2 = in2.rename(columns={"n":"count_dev_lang"})
#ins_dev_lang = in2[['ref_hash','count_dev_lang',]]
#ins_dev_lang.head()

#ins_dev_lang = installs.merge(ins_dev_lang,how='inner',on='ref_hash')
#ins_dev_lang = ins_dev_lang[['ref_hash','count_dev_lang']]

#print("Cantidad ins_dev_lang: "+str(ins_dev_lang['ref_hash'].count()))

#ins_dev_lang.head()

# Armo los features finales para installs

In [73]:
print("Cantidad inst_kind_train: "+str(inst_kind_train['ref_hash'].count()))
inst_kind_train.head(2)

Cantidad inst_kind_train: 480566


,ref_hash,count_for_kind
0,4432995619177048534,377704.0
1,5904733559638204455,377704.0


In [78]:
print("Cantidad in_win_f: "+str(in_win_f['ref_hash'].count()))
in_win_f.head(2)

Cantidad in_win_f: 144845


,ref_hash,window_nr,win_2_in_1,win_3_in_2
0,90072729247980,3,False,False
1,342614246084071,3,False,False


In [75]:
print("Cantidad inst_app_count: "+str(inst_app_count['ref_hash'].count()))
inst_app_count.head(2)

Cantidad inst_app_count: 408055


,ref_hash,count_app_dif,window_nr
247023,5446085605337844584,10,2
247027,5446085605337844584,10,4


In [77]:
print("Cantidad inst_kind_train: "+str(inst_kind_train['ref_hash'].count()))
inst_kind_train.head(2)

Cantidad inst_kind_train: 480566


,ref_hash,count_for_kind
0,4432995619177048534,377704.0
1,5904733559638204455,377704.0


In [ ]:
#Agregamos features
columns_to_xgboost_installs = ['ref_hash','source_id'\
                      ,'ref_type','secs_to_next_mean'\
                      ,'date_int','next_date_int','secs_to_next','alta_demanda','pico_demanda','is_week']